# Python API for DolphinDB

DolphinDB Python API supports Python 3.6 or above.

DolphinDB Python API in essense encapsulates a subset of the DolphinDB script language. It converts Python script to DolphinDB script to be executed on the DolphinDB server. The execution result can either be saved to an object on DolphinDB server, or be serialized to a Python client object. 

There are 2 types of Python API methods. The first type generates DolphinDB script but does not trigger script execution; the second type triggers script execution. Most Python API methods are of the first type. The table below lists all the methods of the second type. They all belong to the table class. 

| Method        | Explanation          |
|-------------- |--------------|
|connect(host, port, [username, password])    | Connect a session to a DolphinDB server |
|toDF()    | Convert a DolphinDB table object to a pandas Dataframe object |
|executeAs(tableName)    | Save a table on DolphinDB server with the specified table name |
|execute()    | Used with **update** and **delete** methods |
|database(dbPath, ......)    | Create or reload a database |
|dropDatabase(dbPath)    | Delete a database |
|dropPartition(dbPath, partitionPaths)    | Delete database partitions |
|dropTable(dbPath, tableName)    | Delete a table |
|drop(colNameList)    | Delete columns from a table |
|ols(Y, X, intercept)   | Conduct ordinary least squares regression and return a dictionary of regression results |

The examples in this tutorial use a csv file: [example.csv](https://github.com/dolphindb/api-python3/blob/master/data/example.csv).

### 1 Connect to DolphinDB

Python interacts with DolphinDB through a session. In the following example, we first create a session in Python, then connect the session to a DolphinDB server with specified domain name/IP address and port number. We need to start a DolphinDB server before running the following Python script.

In [1]:
import dolphindb as ddb
s = ddb.session()
# for atandalone mode
#s.connect("localhost",8848)

If you need to enter username and password:

In [ ]:
s.connect("localhost",8848, YOUR_USERNAME, YOUR_PASSWORD)

In [2]:
s.connect("localhost",8941,"admin","123456")

True

The default admin account username and password for DolphinDB server are "admin" and "123456" respectively.

### 2 Import data to DolphinDB server

There are 3 types of DolphinDB databases based on where they are saved: in DFS (Distributed File System), in local file system and in memory. DolphinDB is a distributed database system and achieves optimal performance in DFS mode. DFS also automatically manages data storage and duplication. Therefore, we highly recommend to use DFS mode. Please refer to the tutorial multi_machine_cluster_deploy for details. For users to get started quickly, we also use databases in the local file system in the examples in this tutorial. All DFS database paths start with "dfs://".

#### 2.1 Import data as an in-memory table

To import text files into DolphinDB as an in-memory table, use session method **loadText**. It returns a DolphinDB table object in Python, which corresponds to an in-memory table on the DolphinDB server. The DolphinDB table object in Python has a method **toDF** to convert it to a pandas DataFrame.

Please note that to use method **loadText** to load a text file as an in-memory table, data size must be smaller than available memory.

In [3]:
WORK_DIR = "C:/DolphinDB/Data"

# return a DolphinDB table object in Python
trade=s.loadText(WORK_DIR+"/example.csv")

# convert the imported DolphinDB table object into a pandas DataFrame
df = trade.toDF()
print(df)

select TICKER,date,VOL,PRC,BID,ASK from TMP_TBL_6e1159b6
      TICKER       date       VOL        PRC        BID       ASK
0       AMZN 1997-05-15   6029815   23.50000   23.50000   23.6250
1       AMZN 1997-05-16   1232226   20.75000   20.50000   21.0000
2       AMZN 1997-05-19    512070   20.50000   20.50000   20.6250
3       AMZN 1997-05-20    456357   19.62500   19.62500   19.7500
4       AMZN 1997-05-21   1577414   17.12500   17.12500   17.2500
5       AMZN 1997-05-22    983855   16.75000   16.62500   16.7500
6       AMZN 1997-05-23   1330026   18.00000   18.00000   18.1250
7       AMZN 1997-05-27    726192   19.00000   19.00000   19.1250
8       AMZN 1997-05-28    382132   18.37500   18.37500   18.6250
9       AMZN 1997-05-29    289970   18.06250   18.00000   18.1250
10      AMZN 1997-05-30    216530   18.00000   18.00000   18.1250
11      AMZN 1997-06-02     49764   18.12500   18.12500   18.3750
12      AMZN 1997-06-03     99107   17.75000   17.75000   17.8750
13      AMZN 1997-0

The default delimiter for function **loadText** is comma ",". Other delimiters can also be specified. For example, to import a tabular text file:

In [ ]:
t1=s.loadText(WORK_DIR+"/t1.tsv", '\t')

#### 2.2 Import data into a partitioned database

To load data files that are larger than available memory into DolphinDB, we can load data into a partitioned database.

#### 2.2.1 Create a partitioned database

After we create a partitioned database, we cannot modify its partitioning scheme. To make sure the examples below do not use a pre-defined database "valuedb", check if it exists. If it exists, drop it.

In [4]:
if s.existsDatabase(WORK_DIR+"/valuedb"):
    s.dropDatabase(WORK_DIR+"/valuedb")

Now create a value-based partitioned database "valuedb" with a session method **database**. As "example.csv" only has data for 3 stocks, we use a VALUE partition with stock ticker as the partitioning column. The parameter **partitions** indicates the partition scheme.

In [5]:
# 'db' indicates the database handle name on the DolphinDB server.
s.database('db', partitionType=ddb.VALUE, partitions=["AMZN","NFLX","NVDA"], dbPath=WORK_DIR+"/valuedb")
# this is equivalent to executing 'db=database(=WORK_DIR+"/valuedb", VALUE, ["AMZN","NFLX", "NVDA"])' on DolphinDB server.

To create a partitioned database in DFS, just make the database path start with "dfs://". To run the following example, we need to configure a DFS cluster. Please refer to the tutorial [multi_machine_cluster_deploy.md](https://github.com/dolphindb/Tutorials_EN/blob/master/multi_machine_cluster_deploy.md).

In [6]:
s.database('db', partitionType=ddb.VALUE, partitions=["AMZN","NFLX", "NVDA"], dbPath="dfs://valuedb")

In addition to VALUE partition, DolphinDB also supports SEQ, RANGE, LIST, COMBO, and HASH partitions.

#### 2.2.2 Create a partitioned table and append data to the table

After a partitioned database is created successfully, we can import text files to a partitioned table in the partitioned database with function **loadTextEx**. If the partitioned table does not exist, **loadTextEx** creates it and appends the imported data to it. Otherwise, the function appends the imported data to the partitioned table.

In function **loadTextEx**, parameter **dbPath** is the database path; **tableName** is the partitioned table name; **partitionColumns** is the partitioning columns; **filePath** is the absolute path of the text file; **delimiter** is the delimiter of the text file (comma by default).

In the following example, function **loadTextEx** creates a partitioned table **trade** on the DolphinDB server and then appends the data from "example.csv" to the table. 

In [7]:
if s.existsDatabase(WORK_DIR+"/valuedb"):
    s.dropDatabase(WORK_DIR+"/valuedb")
s.database('db', partitionType=ddb.VALUE, partitions=["AMZN","NFLX", "NVDA"], dbPath=WORK_DIR+"/valuedb")
trade = s.loadTextEx("db",  tableName='trade',partitionColumns=["TICKER"], filePath=WORK_DIR + "/example.csv")
print(trade.toDF())

select TICKER,date,VOL,PRC,BID,ASK from TMP_TBL_45472ef4
      TICKER       date       VOL       PRC       BID       ASK
0       AMZN 1997-05-15   6029815   23.5000   23.5000   23.6250
1       AMZN 1997-05-16   1232226   20.7500   20.5000   21.0000
2       AMZN 1997-05-19    512070   20.5000   20.5000   20.6250
3       AMZN 1997-05-20    456357   19.6250   19.6250   19.7500
4       AMZN 1997-05-21   1577414   17.1250   17.1250   17.2500
5       AMZN 1997-05-22    983855   16.7500   16.6250   16.7500
6       AMZN 1997-05-23   1330026   18.0000   18.0000   18.1250
7       AMZN 1997-05-27    726192   19.0000   19.0000   19.1250
8       AMZN 1997-05-28    382132   18.3750   18.3750   18.6250
9       AMZN 1997-05-29    289970   18.0625   18.0000   18.1250
10      AMZN 1997-05-30    216530   18.0000   18.0000   18.1250
11      AMZN 1997-06-02     49764   18.1250   18.1250   18.3750
12      AMZN 1997-06-03     99107   17.7500   17.7500   17.8750
13      AMZN 1997-06-04    257316   17.0000   1

To show the number of rows in the table:

In [8]:
print(trade.rows)

select TICKER,date,VOL,PRC,BID,ASK from TMP_TBL_45472ef4
13136


To show the number of columns in the table:

In [9]:
print(trade.cols)

6


To show the schema of the table:

In [10]:
print(trade.schema)

     name typeString  typeInt
0  TICKER     SYMBOL       17
1    date       DATE        6
2     VOL        INT        4
3     PRC     DOUBLE       16
4     BID     DOUBLE       16
5     ASK     DOUBLE       16


To refer to the table later:

In [11]:
trade = s.loadTable(tableName="trade",dbPath=WORK_DIR+"/valuedb")

#### 2.3 Import data as an in-memory partitioned table

#### 2.3.1 **loadTextEx**

We can import data as an in-memory partitioned table. Operations on an in-memory partitioned table are faster than those on a nonpartitioned in-memory table as the former utilizes parallel computing.

We can use function **loadTextEx** to create an in-memory partitioned database with an empty string for the parameter **dbPath**.

In [12]:
s.database('db', partitionType=ddb.VALUE, partitions=["AMZN","NFLX","NVDA"], dbPath="")

# "dbPath='db'" means that the system uses database handle 'db' to import data into in-memory partitioned table trade
trade=s.loadTextEx(dbPath="db", partitionColumns=["TICKER"], tableName='trade', filePath=WORK_DIR + "/example.csv")

#### 2.3.2 **ploadText**

Function **ploadText** loads a text file in parallel to generate an in-memory partitioned table. It runs much faster than **loadText**.

In [13]:
trade=s.ploadText(WORK_DIR+"/example.csv")
print(trade.rows)

select TICKER,date,VOL,PRC,BID,ASK from TMP_TBL_a1f23718
13136


#### 2.4 Upload data from Python to DolphinDB server

#### 2.4.1 With function **upload**

We can upload a Python object to the DolphinDB server with function **upload**. The input of function **upload** is a Python dictionary object. For this dictionary object, the key is the variable name on DolphinDB server and the value is the Python object.

In [14]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'id': np.int32([1, 2, 3, 4, 3]), 'value':  np.double([7.8, 4.6, 5.1, 9.6, 0.1]), 'x': np.int32([5, 4, 3, 2, 1])})
s.upload({'t1': df})
print(s.run("t1.value.avg()"))

5.44


#### 2.4.2 With function **table**

A DolphinDB table object can be created on the DolphinDB server with the **table** method of a session. The input of the **table** method can be a dictionary, a DataFrame, or a table name on the DolphinDB server. 

In [15]:
# save the table to DolphinDB server as table "test"
dt = s.table(data={'id': [1, 2, 2, 3],
                   'ticker': ['AAPL', 'AMZN', 'AMZN', 'A'],
                   'price': [22, 3.5, 21, 26]}).executeAs("test")

# load table "test" on DolphinDB server 
print(s.loadTable("test").toDF())

select id,ticker,price from TMP_TBL_4d6e3f45
test=(select id,ticker,price from TMP_TBL_4d6e3f45)
select id,ticker,price from test
   id ticker  price
0   1   AAPL   22.0
1   2   AMZN    3.5
2   2   AMZN   21.0
3   3      A   26.0


#### 3 Load DolphinDB database tables

#### 3.1 **loadTable**

To load a table from a database, use function **loadTable**. Parameter **tableName** indicates the partitioned table name; **dbPath** is the database location. If **dbPath** is not specified, **loadTable** will load a DolphinDB table in memory whose name is specified in argument **tableName**.

For a partitioned table: if parameter **memoryMode**=True, load all the data (or selected partitions if parameter **partitions** is specified) of the table into DolphinDB server memory as a partitioned table; if **memoryMode**=false, only load its metadata into DolphinDB server memory. 

#### 3.1.1 Load an entire table

In [16]:
trade = s.loadTable(tableName="trade",dbPath=WORK_DIR+"/valuedb")
trade.toDF()

select TICKER,date,VOL,PRC,BID,ASK from TMP_TBL_0ab8694b


,TICKER,date,VOL,PRC,BID,ASK
0,AMZN,1997-05-15,6029815,23.5000,23.5000,23.6250
1,AMZN,1997-05-16,1232226,20.7500,20.5000,21.0000
2,AMZN,1997-05-19,512070,20.5000,20.5000,20.6250
3,AMZN,1997-05-20,456357,19.6250,19.6250,19.7500
4,AMZN,1997-05-21,1577414,17.1250,17.1250,17.2500
5,AMZN,1997-05-22,983855,16.7500,16.6250,16.7500
6,AMZN,1997-05-23,1330026,18.0000,18.0000,18.1250
7,AMZN,1997-05-27,726192,19.0000,19.0000,19.1250
8,AMZN,1997-05-28,382132,18.3750,18.3750,18.6250
9,AMZN,1997-05-29,289970,18.0625,18.0000,18.1250


#### 3.1.2 Load selected partitions

To load only the "AMZN" partition:

In [17]:
trade = s.loadTable(tableName="trade",dbPath=WORK_DIR+"/valuedb", partitions="AMZN")
print(trade.rows)

select TICKER,date,VOL,PRC,BID,ASK from TMP_TBL_9399d101
4941


#### 3.1.3 Load a partitioned table as in-memory table

In [18]:
trade = s.loadTable(tableName="trade",dbPath=WORK_DIR+"/valuedb", partitions=["NFLX","NVDA"], memoryMode=True)
print(trade.rows)

select TICKER,date,VOL,PRC,BID,ASK from TMP_TBL_f72ea496
8195


#### 3.2. **loadTableBySQL**

Method **loadTableBySQL** imports data from an on-disk partitioned table into a in-memory partitioned table through a SQL query.

In [19]:
import os
if s.existsDatabase(WORK_DIR+"/valuedb"  or os.path.exists(WORK_DIR+"/valuedb")):
    s.dropDatabase(WORK_DIR+"/valuedb")
s.database(dbName='db', partitionType=ddb.VALUE, partitions=["AMZN","NFLX", "NVDA"], dbPath=WORK_DIR+"/valuedb")
t = s.loadTextEx("db",  tableName='trade',partitionColumns=["TICKER"], filePath=WORK_DIR + "/example.csv")

trade = s.loadTableBySQL(tableName="trade", dbPath=WORK_DIR+"/valuedb", sql="select * from trade where date>2010.01.01")
print(trade.rows)

select TICKER,date,VOL,PRC,BID,ASK from trade
5286


#### 4 Databases and Tables

#### 4.1 Database operations

#### 4.1.1 Create a database

To create a partitioned database, use session method **database**.

In [20]:
s.database('db', partitionType=ddb.VALUE, partitions=["AMZN","NFLX", "NVDA"], dbPath=WORK_DIR+"/valuedb")

#### 4.1.2 Delete a database

To delete a database, use session method **dropDatabase**. The following statement will drop a database if it exists.

In [21]:
if s.existsDatabase(WORK_DIR+"/valuedb"):
    s.dropDatabase(WORK_DIR+"/valuedb")

#### 4.1.3 Drop a DFS database partition

To drop a DFS database partition, use session method **dropPartition**.

In [22]:
if s.existsDatabase("dfs://valuedb"):
    s.dropDatabase("dfs://valuedb")
s.database('db', partitionType=ddb.VALUE, partitions=["AMZN","NFLX", "NVDA"], dbPath="dfs://valuedb")
trade=s.loadTextEx(dbPath="dfs://valuedb", partitionColumns=["TICKER"], tableName='trade', filePath=WORK_DIR + "/example.csv")
print(trade.rows)

select TICKER,date,VOL,PRC,BID,ASK from trade
13136


In [23]:
s.dropPartition("dfs://valuedb", partitionPaths=["/AMZN", "/NFLX"])
trade = s.loadTable(tableName="trade", dbPath="dfs://valuedb")
print(trade.rows)

select TICKER,date,VOL,PRC,BID,ASK from TMP_TBL_d60f1baf
4516


In [24]:
print(trade.select("distinct TICKER").toDF())

select distinct TICKER from TMP_TBL_d60f1baf
  distinct_TICKER
0            NVDA


#### 4.2 Table operations

#### 4.2.1 Load a database table

Please see section 3.1. 

#### 4.2.2 Append to a table

The following example appends to a partitioned table on disk. The appending changes the table on disk. To use the appended table, we need to load the table after appending. 

In [27]:
if s.existsDatabase(WORK_DIR+"/valuedb"):
    s.dropDatabase(WORK_DIR+"/valuedb")
s.database('db', partitionType=ddb.VALUE, partitions=["AMZN","NFLX", "NVDA"], dbPath=WORK_DIR+"/valuedb")
trade = s.loadTextEx("db",  tableName='trade',partitionColumns=["TICKER"], filePath=WORK_DIR + "/example.csv")
print(trade.rows)

select TICKER,date,VOL,PRC,BID,ASK from TMP_TBL_5ab66fb6
13136


In [28]:
# take the top 10 rows of table "trade" on the DolphinDB server
t = trade.top(10).executeAs("top10")

trade.append(t)

# table "trade" needs to be reloaded in order to see the appended records
trade = s.loadTable(tableName="trade",dbPath=WORK_DIR+"/valuedb")
print (trade.rows)

select top 10 TICKER,date,VOL,PRC,BID,ASK from TMP_TBL_5ab66fb6
top10=(select top 10 TICKER,date,VOL,PRC,BID,ASK from TMP_TBL_5ab66fb6)
select TICKER,date,VOL,PRC,BID,ASK from TMP_TBL_c0a4567f
13146


The following example appends to an in-memory table. 

In [29]:
trade=s.loadText(WORK_DIR+"/example.csv")
t = trade.top(10).executeAs("top10")
t1=trade.append(t)

print(t1.rows)

select top 10 TICKER,date,VOL,PRC,BID,ASK from TMP_TBL_cbb4ab28
top10=(select top 10 TICKER,date,VOL,PRC,BID,ASK from TMP_TBL_cbb4ab28)
select TICKER,date,VOL,PRC,BID,ASK from TMP_TBL_cbb4ab28
13146


#### 4.3 Update a table

Function **update** can only be used on in-memory tables and must be followed by function **execute**. 

In [30]:
trade = s.loadTable(tableName="trade", dbPath=WORK_DIR+"/valuedb", memoryMode=True)
trade = trade.update(["VOL"],["999999"]).where("TICKER=`AMZN").where(["date=2015.12.16"]).execute()
t1=trade.where("ticker=`AMZN").where("VOL=999999")
print(t1.toDF())

update TMP_TBL_1f59fd5b set VOL=999999
select TICKER,date,VOL,PRC,BID,ASK from TMP_TBL_1f59fd5b where ticker=`AMZN and VOL=999999
     TICKER       date     VOL        PRC        BID        ASK
0      AMZN 1997-05-15  999999   23.50000   23.50000   23.62500
1      AMZN 1997-05-16  999999   20.75000   20.50000   21.00000
2      AMZN 1997-05-19  999999   20.50000   20.50000   20.62500
3      AMZN 1997-05-20  999999   19.62500   19.62500   19.75000
4      AMZN 1997-05-21  999999   17.12500   17.12500   17.25000
5      AMZN 1997-05-22  999999   16.75000   16.62500   16.75000
6      AMZN 1997-05-23  999999   18.00000   18.00000   18.12500
7      AMZN 1997-05-27  999999   19.00000   19.00000   19.12500
8      AMZN 1997-05-28  999999   18.37500   18.37500   18.62500
9      AMZN 1997-05-29  999999   18.06250   18.00000   18.12500
10     AMZN 1997-05-30  999999   18.00000   18.00000   18.12500
11     AMZN 1997-06-02  999999   18.12500   18.12500   18.37500
12     AMZN 1997-06-03  999999   17.75

#### 4.4 Delete records from a table

Function **delete** must be followed by function **execute** to delete records from a table.

In [31]:
trade = s.loadTable(tableName="trade", dbPath=WORK_DIR+"/valuedb", memoryMode=True)
trade.delete().where('date<2013.01.01').execute()
print(trade.rows)

select TICKER,date,VOL,PRC,BID,ASK from TMP_TBL_6384cd5c
3024


#### 4.5 Delete table columns

In [32]:
trade = s.loadTable(tableName="trade", dbPath=WORK_DIR + "/valuedb", memoryMode=True)
t1=trade.drop(['ask', 'bid'])
print(t1.top(5).toDF())

select top 5 TICKER,date,VOL,PRC from TMP_TBL_3a01e960
  TICKER       date      VOL     PRC
0   AMZN 1997-05-15  6029815  23.500
1   AMZN 1997-05-16  1232226  20.750
2   AMZN 1997-05-19   512070  20.500
3   AMZN 1997-05-20   456357  19.625
4   AMZN 1997-05-21  1577414  17.125


#### 4.6 Drop a table

In [ ]:
s.dropTable(WORK_DIR + "/valuedb", "trade")

#### 5 SQL query

DolphinDB's table class supports method chaining to generate SQL statements.

#### 5.1 **select**

#### 5.1.1 A list of column names as input

In [33]:
trade = s.loadTable(tableName="trade",dbPath=WORK_DIR+"/valuedb", memoryMode=True)
print(trade.select(['ticker','date','bid','ask','prc','vol']).toDF())

select ticker,date,bid,ask,prc,vol from TMP_TBL_2503c622
      ticker       date       bid       ask       prc       vol
0       AMZN 1997-05-15   23.5000   23.6250   23.5000   6029815
1       AMZN 1997-05-16   20.5000   21.0000   20.7500   1232226
2       AMZN 1997-05-19   20.5000   20.6250   20.5000    512070
3       AMZN 1997-05-20   19.6250   19.7500   19.6250    456357
4       AMZN 1997-05-21   17.1250   17.2500   17.1250   1577414
5       AMZN 1997-05-22   16.6250   16.7500   16.7500    983855
6       AMZN 1997-05-23   18.0000   18.1250   18.0000   1330026
7       AMZN 1997-05-27   19.0000   19.1250   19.0000    726192
8       AMZN 1997-05-28   18.3750   18.6250   18.3750    382132
9       AMZN 1997-05-29   18.0000   18.1250   18.0625    289970
10      AMZN 1997-05-30   18.0000   18.1250   18.0000    216530
11      AMZN 1997-06-02   18.1250   18.3750   18.1250     49764
12      AMZN 1997-06-03   17.7500   17.8750   17.7500     99107
13      AMZN 1997-06-04   17.0000   17.2500   1

We can use the **showSQL** method to display the SQL statement.

In [34]:
print(trade.select(['ticker','date','bid','ask','prc','vol']).where("date=2012.09.06").where("vol<10000000").showSQL())

select ticker,date,bid,ask,prc,vol from TMP_TBL_2503c622 where date=2012.09.06 and vol<10000000
select ticker,date,bid,ask,prc,vol from TMP_TBL_2503c622 where date=2012.09.06 and vol<10000000


#### 5.1.2 String as input

In [35]:
print(trade.select("ticker,date,bid,ask,prc,vol").where("date=2012.09.06").where("vol<10000000").toDF())

select ticker,date,bid,ask,prc,vol from TMP_TBL_2503c622 where date=2012.09.06 and vol<10000000
  ticker       date        bid     ask     prc      vol
0   AMZN 2012-09-06  251.42999  251.56  251.38  5657816
1   NFLX 2012-09-06   56.65000   56.66   56.65  5368963


#### 5.2 **top**

Get the top records in a table.

In [36]:
trade = s.loadTable(tableName="trade",dbPath=WORK_DIR+"/valuedb")
trade.top(5).toDF()

select top 5 TICKER,date,VOL,PRC,BID,ASK from TMP_TBL_96f449fc


,TICKER,date,VOL,PRC,BID,ASK
0,AMZN,1997-05-15,6029815,23.500,23.500,23.625
1,AMZN,1997-05-16,1232226,20.750,20.500,21.000
2,AMZN,1997-05-19,512070,20.500,20.500,20.625
3,AMZN,1997-05-20,456357,19.625,19.625,19.750
4,AMZN,1997-05-21,1577414,17.125,17.125,17.250


#### 5.3 **where**

We can use **where** method to filter the selection.

#### 5.3.1 method chaining

We can use method chaining to apply multiple conditions.

In [37]:
trade = s.loadTable(tableName="trade",dbPath=WORK_DIR+"/valuedb", memoryMode=True)

# use chaining WHERE conditions and save result to DolphinDB server variable "t1" through function "executeAs"
t1=trade.select(['date','bid','ask','prc','vol']).where('TICKER=`AMZN').where('bid!=NULL').where('ask!=NULL').where('vol>10000000').sort('vol desc').executeAs("t1")
print(t1.toDF())

select date,bid,ask,prc,vol from TMP_TBL_d9a85fe4 where TICKER=`AMZN and bid!=NULL and ask!=NULL and vol>10000000 order by vol desc
t1=(select date,bid,ask,prc,vol from TMP_TBL_d9a85fe4 where TICKER=`AMZN and bid!=NULL and ask!=NULL and vol>10000000 order by vol desc)
select date,bid,ask,prc,vol from t1
          date        bid        ask        prc        vol
0   2007-04-25   56.80000   56.81000   56.81000  104463043
1   1999-09-29   80.75000   80.81250   80.75000   80380734
2   2006-07-26   26.17000   26.18000   26.26000   76996899
3   2007-04-26   62.77000   62.83000   62.78100   62451660
4   2005-02-03   35.74000   35.73000   35.75000   60580703
5   2007-07-25   86.15000   86.18000   86.17990   60565477
6   2009-10-23  118.51000  118.52000  118.49000   59460785
7   2005-11-18   47.95000   47.90000   47.98000   59374260
8   2002-01-22   12.60000   12.61000   12.60000   57235489
9   2000-06-23   33.87500   33.93750   33.87500   52221978
10  2001-11-26   12.16000   12.20000   12.2100

In [38]:
print(t1.rows)

select date,bid,ask,prc,vol from t1
765


We can use the **showSQL** method to display the SQL statement.

In [39]:
print(trade.select(['date','bid','ask','prc','vol']).where('TICKER=`AMZN').where('bid!=NULL').where('ask!=NULL').where('vol>10000000').sort('vol desc').showSQL())

select date,bid,ask,prc,vol from TMP_TBL_d9a85fe4 where TICKER=`AMZN and bid!=NULL and ask!=NULL and vol>10000000 order by vol desc
select date,bid,ask,prc,vol from TMP_TBL_d9a85fe4 where TICKER=`AMZN and bid!=NULL and ask!=NULL and vol>10000000 order by vol desc


#### 5.3.2 Use string as input

We can pass a list of field names as a string to **select** method and conditions as string to **where** method.

In [40]:
trade = s.loadTable(tableName="trade",dbPath=WORK_DIR+"/valuedb")
print(trade.select("ticker, date, vol").where("bid!=NULL, ask!=NULL, vol>50000000").toDF())

select ticker, date, vol from TMP_TBL_5647b914 where bid!=NULL, ask!=NULL, vol>50000000
   ticker       date        vol
0    AMZN 1999-09-29   80380734
1    AMZN 2000-06-23   52221978
2    AMZN 2001-11-26   51543686
3    AMZN 2002-01-22   57235489
4    AMZN 2005-02-03   60580703
5    AMZN 2005-11-18   59374260
6    AMZN 2006-07-26   76996899
7    AMZN 2007-04-25  104463043
8    AMZN 2007-04-26   62451660
9    AMZN 2007-04-27   50795877
10   AMZN 2007-07-25   60565477
11   AMZN 2009-10-23   59460785
12   NFLX 2015-07-16   63461015
13   NFLX 2015-08-24   59952448
14   NFLX 2016-01-20   53009419
15   NFLX 2016-04-19   55728765
16   NFLX 2016-07-19   55685209
17   NVDA 2003-05-09   77604776
18   NVDA 2008-02-14   54612850
19   NVDA 2008-05-09   50758761
20   NVDA 2008-07-03   74521639
21   NVDA 2008-07-07   54965504
22   NVDA 2008-08-13   52383250
23   NVDA 2009-05-08   73520286
24   NVDA 2009-12-07   68570708
25   NVDA 2010-05-14   62518368
26   NVDA 2010-07-29   66732862
27   NVDA 2010-1

#### 5.4 **groupby**

Method **groupby** must be followed by an aggregate function such as **count**, **sum**, **avg**, **std**, etc.

In [42]:
if s.existsDatabase(WORK_DIR+"/valuedb"):
    s.dropDatabase(WORK_DIR+"/valuedb")
s.database('db', partitionType=ddb.VALUE, partitions=["AMZN","NFLX", "NVDA"], dbPath=WORK_DIR+"/valuedb")
trade = s.loadTextEx("db",  tableName='trade',partitionColumns=["TICKER"], filePath=WORK_DIR + "/example.csv")
print(trade.select('count(*)').groupby(['ticker']).sort(bys=['ticker desc']).toDF())

select count(*) from TMP_TBL_53843eb3 group by ticker order by ticker desc
select count(*) from TMP_TBL_53843eb3 group by ticker order by ticker desc
  ticker  count
0   NVDA   4516
1   NFLX   3679
2   AMZN   4941


Calculate the sum of columns "vol" and "prc" in "ticker" groups:

In [43]:
trade = s.loadTable(tableName="trade",dbPath=WORK_DIR+"/valuedb")
print(trade.select(['vol','prc']).groupby(['ticker']).sum().toDF())

select sum(vol),sum(prc) from TMP_TBL_c260d6c5 group by ticker
  ticker      sum_vol       sum_prc
0   AMZN  33706396492  772503.81377
1   NFLX  14928048887  421568.81674
2   NVDA  46879603806  127139.51092


**groupby** can be used with with **having**:

In [44]:
trade = s.loadTable(tableName="trade",dbPath=WORK_DIR+"/valuedb")
print(trade.select('count(ask)').groupby(['vol']).having('count(ask)>1').toDF())

select count(ask) from TMP_TBL_32800c60 group by vol having count(ask)>1
       vol  count_ask
0   579392          2
1  3683504          2
2  5732076          2
3  6299736          2
4  6438038          2
5  6946976          2
6  8160197          2
7  8924303          2


#### 5.5 **contextby**

**contextby** is similar to **groupby** except that for each group, **groupby** returns a scalar but **contextby** returns a vector of the same size as the group.

In [45]:
df= s.loadTable(tableName="trade",dbPath=WORK_DIR+"/valuedb").contextby('ticker').top(3).toDF()
print(df)

select top 3 TICKER,date,VOL,PRC,BID,ASK from TMP_TBL_8d3cb617 context by ticker
  TICKER       date      VOL      PRC      BID      ASK
0   AMZN 1997-05-15  6029815  23.5000  23.5000  23.6250
1   AMZN 1997-05-16  1232226  20.7500  20.5000  21.0000
2   AMZN 1997-05-19   512070  20.5000  20.5000  20.6250
3   NFLX 2002-05-23  7507079  16.7500  16.7500  16.8500
4   NFLX 2002-05-24   797783  16.9400  16.9400  16.9500
5   NFLX 2002-05-28   474866  16.2000  16.2000  16.3700
6   NVDA 1999-01-22  5702636  19.6875  19.6250  19.6875
7   NVDA 1999-01-25  1074571  21.7500  21.7500  21.8750
8   NVDA 1999-01-26   719199  20.0625  20.0625  20.1250


In [47]:
df= s.loadTable(tableName="trade",dbPath=WORK_DIR+"/valuedb").select("TICKER, month(date) as month, cumsum(VOL)").contextby("TICKER,month(date)").toDF()
print(df)

select TICKER, month(date) as month, cumsum(VOL) from TMP_TBL_7c6571df context by TICKER,month(date)
select TICKER, month(date) as month, cumsum(VOL) from TMP_TBL_7c6571df context by TICKER,month(date)
      TICKER      month  cumsum_VOL
0       AMZN 1997-05-01     6029815
1       AMZN 1997-05-01     7262041
2       AMZN 1997-05-01     7774111
3       AMZN 1997-05-01     8230468
4       AMZN 1997-05-01     9807882
5       AMZN 1997-05-01    10791737
6       AMZN 1997-05-01    12121763
7       AMZN 1997-05-01    12847955
8       AMZN 1997-05-01    13230087
9       AMZN 1997-05-01    13520057
10      AMZN 1997-05-01    13736587
11      AMZN 1997-06-01       49764
12      AMZN 1997-06-01      148871
13      AMZN 1997-06-01      406187
14      AMZN 1997-06-01      879488
15      AMZN 1997-06-01     1531229
16      AMZN 1997-06-01     1727832
17      AMZN 1997-06-01     2183363
18      AMZN 1997-06-01     2282844
19      AMZN 1997-06-01     2419305
20      AMZN 1997-06-01     2477363
21    

In [48]:
df= s.loadTable(tableName="trade",dbPath=WORK_DIR+"/valuedb").select("TICKER, month(date) as month, sum(VOL)").contextby("TICKER,month(date)").toDF()
print(df)

select TICKER, month(date) as month, sum(VOL) from TMP_TBL_5c8d92b1 context by TICKER,month(date)
select TICKER, month(date) as month, sum(VOL) from TMP_TBL_5c8d92b1 context by TICKER,month(date)
      TICKER      month    sum_VOL
0       AMZN 1997-05-01   13736587
1       AMZN 1997-05-01   13736587
2       AMZN 1997-05-01   13736587
3       AMZN 1997-05-01   13736587
4       AMZN 1997-05-01   13736587
5       AMZN 1997-05-01   13736587
6       AMZN 1997-05-01   13736587
7       AMZN 1997-05-01   13736587
8       AMZN 1997-05-01   13736587
9       AMZN 1997-05-01   13736587
10      AMZN 1997-05-01   13736587
11      AMZN 1997-06-01    4445036
12      AMZN 1997-06-01    4445036
13      AMZN 1997-06-01    4445036
14      AMZN 1997-06-01    4445036
15      AMZN 1997-06-01    4445036
16      AMZN 1997-06-01    4445036
17      AMZN 1997-06-01    4445036
18      AMZN 1997-06-01    4445036
19      AMZN 1997-06-01    4445036
20      AMZN 1997-06-01    4445036
21      AMZN 1997-06-01    4445036

In [49]:
df= s.loadTable(dbPath=WORK_DIR+"/valuedb", tableName="trade").contextby('ticker').having("sum(VOL)>40000000000").toDF()
print(df)

select TICKER,date,VOL,PRC,BID,ASK from TMP_TBL_88c0cd00 context by ticker having sum(VOL)>40000000000
     TICKER       date       VOL       PRC       BID       ASK
0      NVDA 1999-01-22   5702636   19.6875   19.6250   19.6875
1      NVDA 1999-01-25   1074571   21.7500   21.7500   21.8750
2      NVDA 1999-01-26    719199   20.0625   20.0625   20.1250
3      NVDA 1999-01-27    510637   20.0000   19.8750   20.0000
4      NVDA 1999-01-28    476094   19.9375   19.8750   20.0000
5      NVDA 1999-01-29    509718   19.0000   19.0000   19.3125
6      NVDA 1999-02-01    324104   19.3750   19.3750   19.5000
7      NVDA 1999-02-02    552383   17.8750   17.7500   17.8750
8      NVDA 1999-02-03    157385   18.2500   18.2500   18.5000
9      NVDA 1999-02-04    382986   19.2500   19.0000   19.3125
10     NVDA 1999-02-05    286134   19.8125   19.8125   19.8750
11     NVDA 1999-02-08    321818   19.1250   19.1250   19.1875
12     NVDA 1999-02-09    181793   18.3750   18.3750   18.7500
13     NVDA 199

#### 5.6 Table join

DolphinDB table class has method **merge** for inner, left, and outer join; method **merge_asof** for asof join; method **merge_window** for window join.

#### 5.6.1 **merge**

Specify joining columns with parameter **on** if joining column names are identical in both tables; use parameters **left_on** and **right_on** when joining column names are different. The optional parameter **how** indicates table join type. The default table join mode is inner join. 

In [50]:
trade = s.loadTable(dbPath=WORK_DIR+"/valuedb", tableName="trade")
t1 = s.table(data={'TICKER': ['AMZN', 'AMZN', 'AMZN'], 'date': ['2015.12.31', '2015.12.30', '2015.12.29'], 'open': [695, 685, 674]})
print(trade.merge(t1,on=["TICKER","date"]).toDF())

select * from ej(TMP_TBL_6461ef1d,TMP_TBL_dd21b9a5,`TICKER`date,`TICKER`date)
  TICKER       date      VOL        PRC        BID        ASK  open
0   AMZN 2015-12-29  5734996  693.96997  693.96997  694.20001   674
1   AMZN 2015-12-30  3519303  689.07001  689.07001  689.09998   685
2   AMZN 2015-12-31  3749860  675.89001  675.85999  675.94000   695


We need to specify arguments **left_on** and **right_on** when joining column names are different. 

In [51]:
trade = s.loadTable(dbPath=WORK_DIR+"/valuedb", tableName="trade")
t1 = s.table(data={'TICKER1': ['AMZN', 'AMZN', 'AMZN'], 'date1': ['2015.12.31', '2015.12.30', '2015.12.29'], 'open': [695, 685, 674]})
print(trade.merge(t1,left_on=["TICKER","date"], right_on=["TICKER1","date1"]).toDF())

select * from ej(TMP_TBL_4f5a07cb,TMP_TBL_f61b412c,`TICKER`date,`TICKER1`date1)
  TICKER       date      VOL        PRC        BID        ASK  open
0   AMZN 2015-12-29  5734996  693.96997  693.96997  694.20001   674
1   AMZN 2015-12-30  3519303  689.07001  689.07001  689.09998   685
2   AMZN 2015-12-31  3749860  675.89001  675.85999  675.94000   695


To conduct left join, set parameter **how** to "left". 

In [52]:
trade = s.loadTable(dbPath=WORK_DIR+"/valuedb", tableName="trade")
t1 = s.table(data={'TICKER': ['AMZN', 'AMZN', 'AMZN'], 'date': ['2015.12.31', '2015.12.30', '2015.12.29'], 'open': [695, 685, 674]})
print(trade.merge(t1,how="left", on=["TICKER","date"]).where('TICKER=`AMZN').where('2015.12.23<=date<=2015.12.31').toDF())

select * from lj(TMP_TBL_1cea5291,TMP_TBL_8c54f577,`TICKER`date,`TICKER`date) where TICKER=`AMZN and 2015.12.23<=date<=2015.12.31
  TICKER       date      VOL        PRC        BID        ASK   open
0   AMZN 2015-12-23  2722922  663.70001  663.48999  663.71002    NaN
1   AMZN 2015-12-24  1092980  662.78998  662.56000  662.79999    NaN
2   AMZN 2015-12-28  3783555  675.20001  675.00000  675.21002    NaN
3   AMZN 2015-12-29  5734996  693.96997  693.96997  694.20001  674.0
4   AMZN 2015-12-30  3519303  689.07001  689.07001  689.09998  685.0
5   AMZN 2015-12-31  3749860  675.89001  675.85999  675.94000  695.0


To conduct outer join, set parameter **how** to "outer". A partitioned table can only be outer joined with a partitioned table. An in-memory table can only be outer joined with an in-memory table.

In [53]:
t1 = s.table(data={'TICKER': ['AMZN', 'AMZN', 'NFLX'], 'date': ['2015.12.29', '2015.12.30', '2015.12.31'], 'open': [674, 685, 942]})
t2 = s.table(data={'TICKER': ['AMZN', 'NFLX', 'NFLX'], 'date': ['2015.12.29', '2015.12.30', '2015.12.31'], 'close': [690, 936, 951]})
print(t1.merge(t2, how="outer", on=["TICKER","date"]).toDF())

select * from fj(TMP_TBL_189558ee,TMP_TBL_628cee9c,`TICKER`date,`TICKER`date)
  TICKER        date   open TMP_TBL_628cee9c_TICKER TMP_TBL_628cee9c_date  \
0   AMZN  2015.12.29  674.0                    AMZN            2015.12.29   
1   AMZN  2015.12.30  685.0                                                 
2   NFLX  2015.12.31  942.0                    NFLX            2015.12.31   
3                       NaN                    NFLX            2015.12.30   

   close  
0  690.0  
1    NaN  
2  951.0  
3  936.0  


#### 5.6.2 **merge_asof**

The asof join function is used in non-synchronous join. It is similar to the left join function witht the following differences:
1. The data type of the last matching column is usually temporal. For a row in the left table with time t, if there is not a match of left join in the right table, the row in the right table that corresponds to the most recent time before time t is taken, if all the other matching columns are matched; if there are more than one matching record in the right table, the last record is taken. 
2. If there is only 1 joining column, the asof join function assumes the right table is sorted on the joining column. If there are multiple joining columns, the asof join function assumes the right table is sorted on the last joining column within each group defined by the other joining columns. The right table does not need to be sorted by the other joining columns. If these conditions are not met, we may see unexpected results. The left table does not need to be sorted. 

For **merge_asof** and **merge_window**, we use data files trades.csv and quotes.csv, which are AAPL and FB trades and quotes data taken from NYSE website. 

In [54]:
WORK_DIR = "C:/DolphinDB/Data"
if s.existsDatabase(WORK_DIR+"/tickDB"):
    s.dropDatabase(WORK_DIR+"/tickDB")
s.database('db', partitionType=ddb.VALUE, partitions=["AAPL","FB"], dbPath=WORK_DIR+"/tickDB")
trades = s.loadTextEx("db",  tableName='trades',partitionColumns=["Symbol"], filePath=WORK_DIR + "/trades.csv")
quotes = s.loadTextEx("db",  tableName='quotes',partitionColumns=["Symbol"], filePath=WORK_DIR + "/quotes.csv")

print(trades.top(5).toDF())

select top 5 Time,Symbol,Trade_Volume,Trade_Price from TMP_TBL_c189075f
              Time Symbol  Trade_Volume  Trade_Price
0  09:30:00.087488   AAPL        370466      117.100
1  09:30:00.087681   AAPL        370466      117.100
2  09:30:00.103645   AAPL           100      117.100
3  09:30:00.213850   AAPL            20      117.100
4  09:30:00.264854   AAPL            17      117.095


In [55]:
print(quotes.where("second(Time)>=09:29:59").top(5).toDF())

select top 5 Time,Symbol,Bid_Price,Bid_Size,Offer_Price,Offer_Size from TMP_TBL_e2374990 where second(Time)>=09:29:59
              Time Symbol  Bid_Price  Bid_Size  Offer_Price  Offer_Size
0  09:29:59.300399   AAPL     117.07         1       117.09           1
1  09:29:59.300954   AAPL     117.07         1       117.09           1
2  09:29:59.301594   AAPL     117.05         1       117.19          10
3  09:30:00.499924   AAPL     117.09        46       117.10           3
4  09:30:00.500005   AAPL     116.86        53       117.37          64


In [56]:
print(trades.merge_asof(quotes,on=["Symbol","Time"]).select(["Symbol","Time","Trade_Volume","Trade_Price","Bid_Price", "Bid_Size","Offer_Price", "Offer_Size"]).top(5).toDF())

select top 5 Symbol,Time,Trade_Volume,Trade_Price,Bid_Price,Bid_Size,Offer_Price,Offer_Size from aj(TMP_TBL_c189075f,TMP_TBL_e2374990,`Symbol`Time,`Symbol`Time)
  Symbol             Time  Trade_Volume  Trade_Price  Bid_Price  Bid_Size  \
0   AAPL  09:30:00.087488        370466      117.100     117.05         1   
1   AAPL  09:30:00.087681        370466      117.100     117.05         1   
2   AAPL  09:30:00.103645           100      117.100     117.05         1   
3   AAPL  09:30:00.213850            20      117.100     117.05         1   
4   AAPL  09:30:00.264854            17      117.095     117.05         1   

   Offer_Price  Offer_Size  
0       117.19          10  
1       117.19          10  
2       117.19          10  
3       117.19          10  
4       117.19          10  


To calculate trading cost:

In [57]:
print(trades.merge_asof(quotes, on=["Symbol","Time"]).select("sum(Trade_Volume*abs(Trade_Price-(Bid_Price+Offer_Price)/2))/sum(Trade_Volume*Trade_Price)*10000 as cost").groupby("Symbol").toDF())

select sum(Trade_Volume*abs(Trade_Price-(Bid_Price+Offer_Price)/2))/sum(Trade_Volume*Trade_Price)*10000 as cost from aj(TMP_TBL_c189075f,TMP_TBL_e2374990,`Symbol`Time,`Symbol`Time) group by Symbol
select sum(Trade_Volume*abs(Trade_Price-(Bid_Price+Offer_Price)/2))/sum(Trade_Volume*Trade_Price)*10000 as cost from aj(TMP_TBL_c189075f,TMP_TBL_e2374990,`Symbol`Time,`Symbol`Time) group by Symbol
  Symbol      cost
0   AAPL  0.899823
1     FB  2.722923


#### 5.6.3 **merge_window**

**merge_window** (window join) is a generalization of asof join. With a window defined by parameters **leftBound** (w1) and **rightBound** (w2), for each row in the left table with the value of the last joining column equal to t, find the rows in the right table with the value of the last joining column between (t+w1) and (t+w2) conditional on all other joining columns are matched, then apply **aggFunctions** to the selected rows in the right table. 

The only difference between window join and prevailing window join is that if the right table doesn't contain a matching value for t+w1 (the left boundary of the window), prevailing window join will fill it with the last value before t+w1 (conditional on all other joining columns are matched), and apply **aggFunctions**. To use prevailing window join, set the parameter **prevailing** to be True. 

In [58]:
print(trades.merge_window(quotes, -5000000000, 0, aggFunctions=["avg(Bid_Price)","avg(Offer_Price)"], on=["Symbol","Time"]).where("Time>=15:59:59").top(10).toDF())

wj(TMP_TBL_c189075f,TMP_TBL_e2374990,-5000000000:0,<[avg(Bid_Price),avg(Offer_Price)]>,`Symbol`Time,`Symbol`Time)
select top 10 * from wj(TMP_TBL_c189075f,TMP_TBL_e2374990,-5000000000:0,<[avg(Bid_Price),avg(Offer_Price)]>,`Symbol`Time,`Symbol`Time) where Time>=15:59:59
              Time Symbol  Trade_Volume  Trade_Price  avg_Bid_Price  \
0  15:59:59.003095   AAPL           250      117.620     117.603714   
1  15:59:59.003748   AAPL           100      117.620     117.603714   
2  15:59:59.011092   AAPL            95      117.620     117.603714   
3  15:59:59.011336   AAPL           200      117.620     117.603714   
4  15:59:59.022841   AAPL           144      117.610     117.603689   
5  15:59:59.028169   AAPL           130      117.615     117.603544   
6  15:59:59.035357   AAPL          1101      117.610     117.603544   
7  15:59:59.035360   AAPL           799      117.610     117.603544   
8  15:59:59.035602   AAPL           130      117.610     117.603544   
9  15:59:59.036929  

To calculate trading cost:

In [59]:
trades.merge_window(quotes,-1000000000, 0, aggFunctions="[wavg(Offer_Price, Offer_Size) as Offer_Price, wavg(Bid_Price, Bid_Size) as Bid_Price]", on=["Symbol","Time"], prevailing=True).select("sum(Trade_Volume*abs(Trade_Price-(Bid_Price+Offer_Price)/2))/sum(Trade_Volume*Trade_Price)*10000 as cost").groupby("Symbol").executeAs("tradingCost")

print(s.loadTable(tableName="tradingCost").toDF())

pwj(TMP_TBL_c189075f,TMP_TBL_e2374990,-1000000000:0,<[wavg(Offer_Price, Offer_Size) as Offer_Price, wavg(Bid_Price, Bid_Size) as Bid_Price]>,`Symbol`Time,`Symbol`Time)
select sum(Trade_Volume*abs(Trade_Price-(Bid_Price+Offer_Price)/2))/sum(Trade_Volume*Trade_Price)*10000 as cost from pwj(TMP_TBL_c189075f,TMP_TBL_e2374990,-1000000000:0,<[wavg(Offer_Price, Offer_Size) as Offer_Price, wavg(Bid_Price, Bid_Size) as Bid_Price]>,`Symbol`Time,`Symbol`Time) group by Symbol
tradingCost=select sum(Trade_Volume*abs(Trade_Price-(Bid_Price+Offer_Price)/2))/sum(Trade_Volume*Trade_Price)*10000 as cost from pwj(TMP_TBL_c189075f,TMP_TBL_e2374990,-1000000000:0,<[wavg(Offer_Price, Offer_Size) as Offer_Price, wavg(Bid_Price, Bid_Size) as Bid_Price]>,`Symbol`Time,`Symbol`Time) group by Symbol
select Symbol,cost from tradingCost
  Symbol      cost
0   AAPL  0.953315
1     FB  1.077876


#### 5.7 **executeAs**

Function **executeAs** saves query result as a table on DolphinDB server. 

In [60]:
trade = s.loadTable(dbPath=WORK_DIR+"/valuedb", tableName="trade")
trade.select(['date','bid','ask','prc','vol']).where('TICKER=`AMZN').where('bid!=NULL').where('ask!=NULL').where('vol>10000000').sort('vol desc').executeAs("AMZN")

select date,bid,ask,prc,vol from TMP_TBL_a1ca3916 where TICKER=`AMZN and bid!=NULL and ask!=NULL and vol>10000000 order by vol desc
AMZN=(select date,bid,ask,prc,vol from TMP_TBL_a1ca3916 where TICKER=`AMZN and bid!=NULL and ask!=NULL and vol>10000000 order by vol desc)


To use the table "AMZN" on DolphinDB server:

In [61]:
t1=s.loadTable(tableName="AMZN")

#### 6 Regression

Method **ols** conducts ordinary least squares regression. It returns a dictionary with regression results.

In [62]:
trade = s.loadTable(tableName="trade",dbPath=WORK_DIR + "/valuedb", memoryMode=True)
z=trade.select(['bid','ask','prc']).ols('PRC', ['BID', 'ASK'])

print(z["ANOVA"])

select bid,ask,prc from TMP_TBL_8a1563fa
olsEx(sqlDS(<select bid,ask,prc from TMP_TBL_8a1563fa>),"PRC",['BID', 'ASK'],true,2)
    Breakdown     DF            SS            MS             F  Significance
0  Regression      2  2.689281e+08  1.344640e+08  1.214740e+10           0.0
1    Residual  13133  1.453740e+02  1.106937e-02           NaN           NaN
2       Total  13135  2.689282e+08           NaN           NaN           NaN


In [63]:
print(z["RegressionStat"])

           item    statistics
0            R2      0.999999
1    AdjustedR2      0.999999
2      StdError      0.105211
3  Observations  13136.000000


In [64]:
print(z["Coefficient"])

      factor      beta  stdError      tstat    pvalue
0  intercept  0.003710  0.001155   3.213187  0.001316
1        BID  0.605307  0.010517  57.552527  0.000000
2        ASK  0.394712  0.010515  37.537919  0.000000


In [65]:
print(z["Coefficient"].beta[1])

0.6053065015003085


The following example conducts regression on a partitioned database. Note that the ratio operator between 2 integer columns in DolphinDB is "\", which happens to be the escape character in Python, so we need to use "VOL\\SHROUT" in function **select**.

In [68]:
# result = s.loadTable(tableName="US",dbPath="dfs://US").select("select VOL\\SHROUT as turnover, abs(RET) as absRet, (ASK-BID)/(BID+ASK)*2 as spread, log(SHROUT*(BID+ASK)/2) as logMV").where("VOL>0").ols("turnover", ["absRet","logMV", "spread"], True)
# print(result["ANOVA"])

#### 7 **run**

A session object has a method **run** that can execute any DolphinDB script. If the script returns an object in DolphinDB, method **run** converts the object to a corresponding object in Python.  

In [69]:
# Load table
trade = s.loadTable(dbPath=WORK_DIR+"/valuedb", tableName="trade")

# query the table and return a pandas DataFrame
t = s.run("select bid, ask, prc from trade where bid!=NULL, ask!=NULL, vol>1000")
print(t)

           bid       ask       prc
0      19.6250   19.6875   19.6875
1      21.7500   21.8750   21.7500
2      20.0625   20.1250   20.0625
3      19.8750   20.0000   20.0000
4      19.8750   20.0000   19.9375
5      19.0000   19.3125   19.0000
6      19.3750   19.5000   19.3750
7      17.7500   17.8750   17.8750
8      18.2500   18.5000   18.2500
9      19.0000   19.3125   19.2500
10     19.8125   19.8750   19.8125
11     19.1250   19.1875   19.1250
12     18.3750   18.7500   18.3750
13     18.1250   18.5000   18.1875
14     19.6250   20.0000   19.7500
15     20.8750   20.9375   20.8750
16     20.2500   21.0000   21.0000
17     19.8750   20.1250   19.8750
18     20.0000   20.1875   20.1875
19     20.8750   21.0000   20.8750
20     21.0000   21.1250   21.0000
21     22.0000   22.3750   22.0000
22     23.6250   23.7500   23.7500
23     22.8750   23.0000   23.0000
24     21.9375   22.0000   21.9375
25     21.7500   22.0625   22.0625
26     21.6875   21.8750   21.8750
27     20.3750   20.